In [25]:
import numpy as np

np.random.seed(123)
X = np.load('../generated_games/features-40k.npy')
Y = np.load('../generated_games/labels-40k.npy')
print("features", X.shape)
print("labels", Y.shape)

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

features (41439, 1, 9, 9)
labels (41439, 81)
X_train (37295, 9, 9, 1)
Y_train (37295, 81)
X_test (4144, 9, 9, 1)
Y_test (4144, 81)


In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(filters=48,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Conv2D(64, (3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten()) 
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax')) 
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 9, 9, 48)       │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 9, 9, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 9, 9, 64)       │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 594,545 (2.27 MB)

 Trainable params: 594,545 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
import time

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=100,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.0165 - loss: 4.3870 - val_accuracy: 0.0208 - val_loss: 4.3110
Epoch 2/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0246 - loss: 4.3166 - val_accuracy: 0.0268 - val_loss: 4.2686
Epoch 3/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0221 - loss: 4.2895 - val_accuracy: 0.0251 - val_loss: 4.2497
Epoch 4/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0220 - loss: 4.2692 - val_accuracy: 0.0290 - val_loss: 4.2367
Epoch 5/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0233 - loss: 4.2523 - val_accuracy: 0.0273 - val_loss: 4.2153
Epoch 6/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0254 - loss: 4.2265 - val_accuracy: 0.0270 - val_loss: 4.1883
Epoch 7/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0250 - loss: 4.1992 - val_accuracy: 0.0275 - val_loss: 4.1612
Epoch 8/100
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0257 - loss: 4.1695 - 

In [ ]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]]).reshape(9, 9)
board_matrix = np.zeros(9, 9, 1)
for r in range(9):
    for c in range(9):
        p = test_board(r + 1, c + 1)
        board_matrix[r + 1, c + 1, 0] = p
# print(test_board)
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=int64)
  • training=False
  • mask=None

Starlight MacBook Air M2:
> Wall time: 738.242098 seconds

> Test loss: 3.81455397605896
> Test accuracy: 0.07915057986974716

MacBookPro M1 Pro:
> Wall time: 38.298259 seconds

> Test loss: 4.374318599700928
> Test accuracy: 0.014076245948672295